## The project for Face,Age,gender,emotion Recognition is starting from now!

###### just in my virtual environment i installed the opencv-contrib-python in oredr to make this project, simply activated the venv by opening the folder from my dekstop where exactly i created the virtual env and by entering the promt "Bristi-virtual-env\Scripts\activate" i activated the env and using "pip install opencv-contrib-python" i installed the opencv package,where opencv-python	Main OpenCV modules (basic functionality)
###### opencv-contrib-python	Main modules + extra modules from the "contrib" repository 

##### now for downloading the dataset go in google search for"haarcascade frontal face xml default " and open the kaggale and by right clicking and saving it it will be downloaded.

### Import Packages

In [20]:
import os # for reading and writing in files
import cv2 # for detecting and recognizing the face in web camera
import numpy as np #for numerical computation
from deepface import DeepFace #DeepFace is a deep learning model used for face recognition and verification by converting faces into numerical embeddings for accurate comparison. It enables tasks like identity matching, emotion, age, and gender detection.


### Create Dataset

In [35]:
data_dir = "face_dataset"
os.makedirs(data_dir, exist_ok=True)

def create_face_dataset(name):
    person_dir = os.path.join(data_dir, name)  # for each person there images will be stored in a directory
    os.makedirs(person_dir, exist_ok=True)

    cap = cv2.VideoCapture(0)
    print("capturing Images.Press'q' to quit")# opencv will capture the video and 0 is for webcam
    count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Cannot Capture Image")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # convert to grayscale for detection
        faces = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")\
                 .detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            count += 1
            face_img = frame[y:y+h, x:x+w]
            face_path = os.path.join(person_dir, f"{name}_{count}.jpg")
            cv2.imwrite(face_path, face_img)

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.imshow("Capture Face In Camera", frame)

            
        if cv2.waitKey(1) & 0xFF == ord('q') or count >= 50:
            break
    cap.release()
    cv2.destroyAllWindows()
    print(f"Saved{count}images for {name}.")
    return 


## Train the Dataset

In [36]:
def train_face_dataset():
    embeddings = {}  # extract all the facial feature and convert it into the numerical values
    for person in os.listdir(data_dir):
        person_dir = os.path.join(data_dir, person)
        if os.path.isdir(person_dir):
            embeddings[person] = []
            for img_name in os.listdir(person_dir):
                img_path = os.path.join(person_dir, img_name)
                try:
                    embedding = DeepFace.represent(img_path, model_name="Facenet", enforce_detection=False)[0]["embedding"]
                    embeddings[person].append(embedding)
                except Exception as e:
                    print(f"Failed to train image: {img_path}:{e}")
    return embeddings


## Recognize the Face,age,gender,emotion using Deepface (Facenet model)

In [37]:
def recognize_faces(embeddings):
    cap = cv2.VideoCapture(0)  

    while True:
        ret, frame = cap.read()
        if not ret:
            print("failed to capture Image")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  

        faces = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")\
            .detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            try:
                analyse = DeepFace.analyze(face_img, actions=["age", "gender", "emotion"], enforce_detection=False)
                if isinstance(analyse, list):
                    analyse = analyse[0]

                age = analyse["age"]

                gender = analyse["dominant_gender"]  

                gender =gender if isinstance(gender,str)else max(gender, key=gender.get)

                emotion = max(analyse["emotion"], key=analyse["emotion"].get)
                #recognize person
                face_embedding = DeepFace.represent(face_img, model_name="Facenet", enforce_detection=False)[0]["embedding"]
                #calculate cosine similarity
                match = None
                max_similarity = -1  

                for person, person_embeddings in embeddings.items():  
                    for embed in person_embeddings:
                        similarity = np.dot(face_embedding, embed) / (np.linalg.norm(face_embedding) * np.linalg.norm(embed))
                        if similarity > max_similarity:
                            max_similarity = similarity
                            match = person

                if max_similarity > 0.7:
                    label = f"{match} ({max_similarity:.2f})" 
                    #print(f"Recognized{label}--Exiting recognition.")
                    #cap.release()
                    #cv2.destroyAllWindows()
                else:
                    label = "Unknown person"

                display_text = f"{label}, Age: {int(age)}, Gender: {gender}, Emotion: {emotion}"
                cv2.putText(frame, display_text, org=(x, y - 10),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5,
                            color=(255, 255, 255), thickness=2)

            except Exception as e:
                print("CAN NOT RECOGNIZE FACES")

        cv2.imshow("Recognize Faces", frame)  

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Pressed Q.Exiting")
            break

    cap.release()
    cv2.destroyAllWindows()  


## OUTPUT

In [38]:
if __name__ == "__main__":
    print("1.Create face Dataset\n2.Train Face Dataset\n3.Recognize faces")
    choice = input("Enter your choice: ")

    if choice == "1":
        name = input("Enter name of the person: ")
        create_face_dataset(name)

    elif choice == "2":
        embeddings = train_face_dataset()
        np.save("embeddings.npy", embeddings)
        print("Embedding Saved")

    elif choice == "3":
        if os.path.exists("embeddings.npy"):
            embeddings = np.load("embeddings.npy", allow_pickle=True).item()
            recognize_faces(embeddings)
        else:
            print("No File is Found")

    else:
        print("Invalid choice")


1.Create face Dataset
2.Train Face Dataset
3.Recognize faces


Enter your choice:  1
Enter name of the person:  Tinku Ghosh


capturing Images.Press'q' to quit
Saved1images for Tinku Ghosh.


In [39]:
import os
print(os.getcwd())

C:\Users\DELL


In [40]:
import os
os.listdir("Dataset")


['Bristi Ghosh']

In [32]:
os.listdir("Dataset/Bristi Ghosh")  # replace with actual name
#This PC > Local Disk (C:) > Users > DELL > Dataset
#this is to find where the dataset is being stored

['Bristi Ghosh_1.jpg',
 'Bristi Ghosh_10.jpg',
 'Bristi Ghosh_11.jpg',
 'Bristi Ghosh_12.jpg',
 'Bristi Ghosh_13.jpg',
 'Bristi Ghosh_14.jpg',
 'Bristi Ghosh_15.jpg',
 'Bristi Ghosh_17.jpg',
 'Bristi Ghosh_18.jpg',
 'Bristi Ghosh_19.jpg',
 'Bristi Ghosh_2.jpg',
 'Bristi Ghosh_20.jpg',
 'Bristi Ghosh_21.jpg',
 'Bristi Ghosh_22.jpg',
 'Bristi Ghosh_23.jpg',
 'Bristi Ghosh_24.jpg',
 'Bristi Ghosh_25.jpg',
 'Bristi Ghosh_26.jpg',
 'Bristi Ghosh_27.jpg',
 'Bristi Ghosh_28.jpg',
 'Bristi Ghosh_29.jpg',
 'Bristi Ghosh_3.jpg',
 'Bristi Ghosh_30.jpg',
 'Bristi Ghosh_31.jpg',
 'Bristi Ghosh_32.jpg',
 'Bristi Ghosh_33.jpg',
 'Bristi Ghosh_34.jpg',
 'Bristi Ghosh_35.jpg',
 'Bristi Ghosh_36.jpg',
 'Bristi Ghosh_37.jpg',
 'Bristi Ghosh_38.jpg',
 'Bristi Ghosh_39.jpg',
 'Bristi Ghosh_4.jpg',
 'Bristi Ghosh_40.jpg',
 'Bristi Ghosh_41.jpg',
 'Bristi Ghosh_42.jpg',
 'Bristi Ghosh_43.jpg',
 'Bristi Ghosh_44.jpg',
 'Bristi Ghosh_45.jpg',
 'Bristi Ghosh_46.jpg',
 'Bristi Ghosh_47.jpg',
 'Bristi Ghosh_48.jp

In [33]:
import os
os.startfile("Dataset")  # will open the folder in File Explorer

